In [2]:
import sys
sys.path.insert(0, '/gpfs/data1/cmongp/Bharath/lib/')

In [3]:
import os
import pandas as pd
import numpy as np

from utils import *

In [4]:
import subprocess

In [13]:
base_dir = '/gpfs/data1/cmongp/rohitN/CSM/Exp4TFPStudy/OK/Testing'
EPIC_csv_filepath = '/gpfs/data1/cmongp/Bharath/OKTPN_1.csv'
SOIL_filepath_master = '/gpfs/data1/cmongp/Bharath/ks_ok_soils.csv'

In [10]:
EPIC_csv = pd.read_csv(EPIC_csv_filepath)


In [17]:
df_ls = EPIC_csv.to_dict('records')

In [5]:
import os
import shutil
from subprocess import call

def process_model(row):
    id = row['OBJECTID']
    # Define paths
    # base_dir = '/gpfs/data1/cmongp/rohitN/CSM/Exp4TFPStudy/OK/Testing'
    model_dir = os.path.join(base_dir, 'EPICModel')
    new_dir = os.path.join(base_dir, str(id))
    output_dir = os.path.join(base_dir, 'out')

    # Delete the new directory if it exists
    if os.path.exists(new_dir):
        shutil.rmtree(new_dir)
        
    # Copy EPICModel to a new directory named as id
    shutil.copytree(model_dir, new_dir)
    
    # Assuming the file to edit is file.txt
    # Modify the file as you need
    with open(f'{new_dir}/EPICRUN.DAT', 'w') as ofile:
        fmt = '%8d %8d %8d 0 1 %8d  %8d  %8d  0   0  25   1995   10.00   2.50  2.50  0.1/'
        np.savetxt(ofile, [[int(id)]*6], fmt=fmt)
        
    with open(f'{new_dir}/ieSite.DAT', 'w') as ofile:
        fmt = '%8d    "../common_inputfiles/site/%d.sit"'
        np.savetxt(ofile, [[row['OBJECTID'], row['base']]], fmt=fmt)

    with open(f'{new_dir}/ieSllist.DAT', 'w') as ofile:
        fmt = '%8d    "../common_inputfiles/soils/%d.SOL"'  
        np.savetxt(ofile, [[row['OBJECTID'], row['ssu']]], fmt=fmt)

    with open(f'{new_dir}/ieWedlst.DAT', 'w') as ofile:
        fmt = '%8d    "../../common_inputfiles/daily/%d.DLY"'  
        np.savetxt(ofile, [[row['OBJECTID'], row['dly']]], fmt=fmt)

    with open(f'{new_dir}/ieWealst.DAT', 'w') as ofile:
        fmt = '%8d    "../../common_inputfiles/monthly/%d.INP"   %.2f   %.2f  NB            XXXX'  
        np.savetxt(ofile, [[row['OBJECTID'], row['dly'], row['x'], row['y']]], fmt=fmt)
    
    with open(f'{new_dir}/ieOplist.DAT', 'w') as ofile:
        fmt = '%8d    "../../common_inputfiles/OPC/CropRot_%d.OPC"'  
        np.savetxt(ofile, [[row['OBJECTID'], row['rotID']]], fmt=fmt)
    
    # Run the simulation from the new directory
    os.chdir(new_dir)
    command = f'nohup ./EPIC1102Versions_20201015 &> {base_dir}/log/{id}.out'
    message = subprocess.Popen(command, shell=True).wait()
    
    #check for error
    with open(f'{base_dir}/log/{id}.out', 'r') as file:
        lines = file.readlines()
        last_line = lines[-1]  # Get the last line
        if last_line.startswith('EPIC'):
            # Save the id into a file
            with open(f'{base_dir}/saved_ids.txt', 'a') as id_file: 
                id_file.write(f'{id}\n') 

    # Copy the id.ACY and id.ACM files to output dir
    shutil.copy(os.path.join(new_dir, f'{id}.ACY'), output_dir)
    shutil.copy(os.path.join(new_dir, f'{id}.ACM'), output_dir)
    shutil.copy(os.path.join(new_dir, f'{id}.ACO'), output_dir)
    
    # Delete the whole new directory
    os.chdir(base_dir)
    shutil.rmtree(new_dir)

In [14]:
# process_model({'OBJECTID': 11871139, 'problty': 100.0, 'rotat': 'Continuous Winter wheat', 'rotID': 6.0, 'x': -97.48982658274372, 'y': 36.39636614597452, 'slope': 0.3059178, 'ele': 338, 'ssu': 382525, 'base': 2054239, 'dly': 400064329, 'id': 192069, 'mukey': 382526.0, 'sl_length': 68, 'slope_steep': 0.0, 'nldas_id': 1516})

In [16]:
parallel_executor(process_model, df_ls, max_workers = 80)

  0%|          | 1/24113 [00:01<11:55:28,  1.78s/it]

Function execution failed for args:
 {'OBJECTID': 11871139, 'problty': 100.0, 'rotat': 'Continuous Winter wheat', 'rotID': 6.0, 'x': -97.48982658274372, 'y': 36.39636614597452, 'slope': 0.3059178, 'ele': 338, 'ssu': 382525, 'base': 2054239, 'dly': 400064329, 'id': 192069, 'mukey': 382526.0, 'sl_length': 68, 'slope_steep': 0.0, 'nldas_id': 1516}
Exception: [Errno 39] Directory not empty: '/gpfs/data1/cmongp/rohitN/CSM/Exp4TFPStudy/OK/Testing/11871139'.
Function execution failed for args:
 {'OBJECTID': 11871139, 'problty': 100.0, 'rotat': 'Continuous Winter wheat', 'rotID': 6.0, 'x': -97.48982658274372, 'y': 36.39636614597452, 'slope': 0.3059178, 'ele': 338, 'ssu': 382525, 'base': 2054239, 'dly': 400064329, 'id': 192069, 'mukey': 382526.0, 'sl_length': 68, 'slope_steep': 0.0, 'nldas_id': 1516}
Exception: [Errno 2] No such file or directory: '/gpfs/data1/cmongp/rohitN/CSM/Exp4TFPStudy/OK/Testing/11871139/11871139.ACY'.


100%|██████████| 24113/24113 [16:25<00:00, 24.46it/s]


([], [1, 0])

In [21]:
failed = pd.read_csv(os.path.join(base_dir, 'saved_ids.txt'))

FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/data1/cmongp/rohitN/CSM/Exp4TFPStudy/OK/Testing/EPICModel/saved_ids.txt'

In [19]:
EPIC_csv.set_index('OBJECTID', inplace = True)

In [19]:
EPIC_csv_failed = EPIC_csv.loc[failed.values.flatten()]

In [22]:
failed_soilids = list(set(EPIC_csv_failed['ssu']))

In [23]:
soil_info_master = pd.read_csv(SOIL_filepath_master)
soil_info_master.set_index('band_1', inplace=True)
valid_soil_ids = soil_info_master.loc[set(soil_info_master.index).difference(failed_soilids)]
invalid_soil_ids = soil_info_master.loc[failed_soilids]
inds = find_nearest(invalid_soil_ids, valid_soil_ids, metric = 'minkowski', k = 1)
invalid_soil_ids['nearest'] = (valid_soil_ids.index.values)[inds]
soil_info_master.loc[invalid_soil_ids.index.values, 'nearest'] = invalid_soil_ids['nearest'].values

In [42]:
# soil_info_master.to_csv('/gpfs/data1/cmongp/Bharath/ks_ok_soils.csv')

In [ ]:
EPIC_csv_failed['ssu'] = EPIC_csv_failed['ssu'].apply(lambda x: soil_info_master.loc[int(x), 'nearest'])

In [44]:
EPIC_csv_failed.to_csv('failing_ids.csv')
EPIC_csv_failed = pd.read_csv('failing_ids.csv')

In [25]:
with open(os.path.join(base_dir, 'failed_soil_ids.txt'), 'w') as f:
    f.write('\n'.join(failed_soilids))
# pd.DataFrame(failed_soilids).to_csv('failing_soil_ids.csv', index = False)

In [ ]:
df_ls = EPIC_csv_failed.to_dict('records')
parallel_executor(process_model, df_ls, max_workers = 80)

In [6]:
pd.read_csv('failing_ids.csv')

,OBJECTID,problty,rotat,rotID,x,y,slope,ele,ssu,base,dly,id,mukey,sl_length,slope_steep,nldas_id
0,11871161,66.666667,Continuous Soybean,2.0,-97.479749,36.396037,0.305918,338,382528,2054242,400064330,192070,382527.0,68,0.00,1516
1,11871222,66.666667,Continuous Winter wheat,6.0,-97.438024,36.392002,0.386217,337,1907378,2056362,400064335,192186,1907375.0,53,0.00,1516
2,11871245,66.666667,Continuous Winter wheat,6.0,-97.436056,36.391653,0.386217,337,1907378,2056363,400064335,192186,1907375.0,53,0.00,1516
3,11871251,85.714286,Continuous Winter wheat,6.0,-97.424345,36.391487,0.791237,338,384069,2056365,400064337,192126,384070.0,68,0.01,1516
4,11871262,100.000000,Continuous Winter wheat,6.0,-97.494402,36.392416,0.598804,336,382489,2056350,400064329,192031,382488.0,33,0.01,1516
5,11871285,80.000000,Continuous Winter wheat,6.0,-97.447176,36.389698,0.712803,339,1907378,2056360,400064334,192186,1907375.0,53,0.01,1516
6,11871287,75.000000,Continuous Winter wheat,6.0,-97.538582,36.389416,0.545207,329,382485,2056341,400065275,192027,382484.0,33,0.01,1515
7,11871294,50.000000,Continuous Winter wheat,6.0,-97.507134,36.389776,0.442062,333,382532,2056348,400064327,192076,382533.0,33,0.00,1515
8,11871312,50.000000,Continuous Winter wheat,6.0,-97.510203,36.388231,0.442062,333,382532,2058459,400065278,192076,382533.0,33,0.00,1515
9,11871329,100.000000,Continuous Winter wheat,6.0,-97.444756,36.388590,0.712803,339,1907378,2058473,400065285,192186,1907375.0,53,0.01,1516
